# Use Spark UDF with Spark SQL and Spark DataFrame

#### Topics covered in this example
* Creating and registering UDF.
* Special handling and best practices for UDF.

***

## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* The EMR cluster attached to this notebook should have the `Spark` application installed.
* This notebook uses the `Spark` kernel.
***

## Introduction
User-Defined Functions (UDFs) are user-programmable routines that act on one row. This example demonstrate how to define and register UDFs and invoke them in SQL using the `%%sql` magic.

#### Why do you need UDFs ?
Spark stores data in dataframes or RDDs—resilient distributed datasets. As with a traditional SQL database, you cannot create your own custom function and run that against the database directly unless you register the function first. That is, save it to the database as if it were one of the built-in database functions, like sum(), average(), count(), etc.

The document: <a href="https://docs.databricks.com/spark/latest/spark-sql/udf-scala.html" target="_blank">Scala UDF</a> provides detailed information.
***

## Example

Create a spark dataFrame.

In [ ]:
import spark.implicits._

val columns = Seq("No", "Name")

val data = Seq(("1", "john jones"),
               ("2", "tracey smith"),
               ("3", "amy sanders"))

val df = data.toDF(columns:_*)

Print the table dataFrame.

In [ ]:
df.show(false)

#### Create Spark UDF to use it on DataFrame

Create a function to convert a string to camel case. The function takes a string parameter and converts the first letter of every word to upper case letter.

In [ ]:
val convertToCamelCase = (str:String) => {
    val arr = str.split(" ")
    arr.map(f =>  f.substring(0,1).toUpperCase + f.substring(1,f.length)).mkString(" ")
}

Now convert this function `convertToCamelCase()` to a UDF by passing the function to Spark SQL `udf()`.

In [ ]:
val convertToCamelCaseUDF = udf(convertToCamelCase)

Now you can use the `convertToCamelCaseUDF()` on a DataFrame column.

In [ ]:
df.select(col("No"), convertToCamelCaseUDF(col("Name")).as("Name") ).show(false)

#### Registering Spark UDF to use it on SQL

In order to use a function on Spark SQL, you need to register the function with Spark using `spark.udf.register()`.

Create another function convertToUpperCase to convert the whole string to upper case and register as udf.

In [ ]:
val convertToUpperCase = (str:String) => {
    str.toUpperCase()
}

spark.udf.register("convertToUpperCaseUDF", convertToUpperCase)

Create a temp view named `NAME_TABLE`. `createOrReplaceTempView` creates (or replaces if that view name already exists) a lazily evaluated view that you can then use like a hive table in Spark SQL. It is not persistent at this moment but you can run SQL queries on top of the view.

In [ ]:
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select No, convertToUpperCaseUDF(Name) from NAME_TABLE").show(false)

You can also use the `%%sql` magic to query. Use the `convertToUpperCaseUDF` udf and display a new column `UpperCaseName`.

In [17]:
%%sql
select No, Name, convertToUpperCaseUDF(Name) as UpperCaseName from NAME_TABLE

### Special Handling

#### Execution order
Spark SQL (including SQL and the DataFrame and Dataset API) does not guarantee the order of evaluation of subexpressions. In particular, the inputs of an operator or function are not necessarily evaluated left-to-right or in any other fixed order. For example, logical AND and OR expressions do not have left-to-right “short-circuiting” semantics.

Therefore, it is dangerous to rely on the side effects or order of evaluation of Boolean expressions, and the order of WHERE and HAVING clauses, since such expressions and clauses can be reordered during query optimization and planning. Specifically, if a UDF relies on short-circuiting semantics in SQL for null checking, there’s no guarantee that the null check will happen before invoking the UDF. For example,

```
%%sql
SELECT No, convertToUpperCaseUDF(Name) as Name from NAME_TABLE WHERE Name is not null and convertToUpperCaseUDF(Name) like "%John%"
```

This WHERE clause does not guarantee the `convertToUpperCaseUDF` to be invoked after filtering out nulls.

#### Handling null check
UDF’s are error-prone when not designed carefully. for example, when you have a column that contains the value null on some records.

```
columns = ["No","Name"]

data = [("1", "john jones"),
        ("2", "tracey smith"),
        ("3", "amy sanders"),
        ("4", null)]

%%sql
select No, convertToUpperCaseUDF(Name) as Name from NAME_TABLE
```

Record with `No 4` has value `null` for the `Name` column. Since we are not handling null with UDF function, using this on DataFrame returns an error.

#### Performance

UDF’s are a black box and hence optimizations can’t be applied on the Dataframe/Dataset. When possible you should use the Spark SQL built-in functions as these functions provide optimizations.

***
### Best practices while using UDFs.

* Make the UDF null-aware and do null checking inside the UDF itself.
* Use `IF` or `CASE WHEN` expressions to do the null check and invoke the UDF in a conditional branch.
* Create UDF only when existing built-in SQL function are insufficient.

***
## Additional Notes
To contribute to this example, please contact the EMR Notebooks team.